In [8]:
import pandas as pd

data = pd.read_csv('data/test.csv')

data.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_1500,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000
0,30474,2015-07-01,39.0,20.7,2,9,1,1998.0,1,8.9,...,8,0,0,0,1,10,1,0,14,1
1,30475,2015-07-01,79.2,NaN,8,17,1,0.0,3,1.0,...,4,1,1,0,2,11,0,1,12,1
2,30476,2015-07-01,40.5,25.1,3,5,2,1960.0,2,4.8,...,42,11,4,0,10,21,0,10,71,11
3,30477,2015-07-01,62.8,36.0,17,17,1,2016.0,2,62.8,...,1,1,2,0,0,10,0,0,2,0
4,30478,2015-07-01,40.0,40.0,17,17,1,0.0,1,1.0,...,5,1,1,0,2,12,0,1,11,1


In [9]:
with open('removed_columns.txt', 'r') as f:
    removed_columns = [line.strip() for line in f.readlines()]

def preprocess(data):

    data = data.drop([
        'ID_metro',
        'ID_railroad_station_walk',
        'ID_railroad_station_avto',
        'ID_big_road1',
        'ID_big_road2',
        'ID_railroad_terminal',
        'ID_bus_terminal'
    ], axis=1)

    data = data.drop('id', axis=1)

    data = data.drop(columns=removed_columns, errors='ignore')

    numeric_columns = data.loc[:, data.dtypes!='object'].columns

    for col in numeric_columns:
        data[col] = data[col].fillna(data[col].mean())

    categorical_columns = data.loc[:, data.dtypes == 'object'].columns

    for col in categorical_columns:
        if col != 'timestamp':
            if data[col].nunique() < 5:
                one_hot = pd.get_dummies(data[col], prefix=col, drop_first=True)
                data = pd.concat((data.drop(col, axis=1), one_hot), axis=1)

            else:
                mean_target = data.groupby(col)['full_sq'].mean()
                data[col] = data[col].map(mean_target)

    data['timestamp'] = pd.to_datetime(data['timestamp'])

    data['month'] = data.timestamp.dt.month
    data['year'] = data.timestamp.dt.year

    data = data.sort_values(['timestamp'])

    one_hot = pd.get_dummies(data['year'], prefix='year', drop_first=True)
    data = pd.concat((data.drop('year', axis=1), one_hot), axis=1)
    data = data.replace({True: 1, False: 0})

    one_hot = pd.get_dummies(data['month'], prefix='month', drop_first=True)
    data = pd.concat((data.drop('month', axis=1), one_hot), axis=1)
    data = data.replace({True: 1, False: 0})

    data = data.drop('timestamp', axis=1)

    Owner_Occupier = data[data['product_type_OwnerOccupier'] == 1].copy()
    Investment = data[data['product_type_OwnerOccupier'] == 0].copy()

data = preprocess(data)

C:\Users\denis\AppData\Local\Temp\ipykernel_4316\3907747967.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({True: 1, False: 0})
C:\Users\denis\AppData\Local\Temp\ipykernel_4316\3907747967.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.replace({True: 1, False: 0})


In [5]:
data.columns()

AttributeError: 'NoneType' object has no attribute 'columns'